## Instalar AdapterHub

La siguiente línea es para descargar e instalar **AdapterHub**.

In [ ]:
#!pip install adapter-transformers

## Librerías

In [1]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import os

## Dataset

In [2]:
# CREATE DATASET CLASS---------------------------------------------------------------------------------------------

import os
from torch.utils.data import Dataset

class polar(Dataset):

  def __init__(self, Dir, split, tokenizer, use_labels=True):
    self.use_labels = use_labels

    csv_file   = os.path.join(Dir, split + '.csv')

    self.df = pd.read_csv(csv_file)

    labels_file = os.path.join(Dir, split + '_labels.txt')
    if self.use_labels:
      self.labels    = self.df['polarization']

    self.texts = self.df['text'].tolist()

    self.encodings = tokenizer(
        self.texts,
        max_length = 128,
        truncation = True,
        padding = True
      )

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['text']   = self.texts[idx]
    if self.use_labels:
      item['labels'] = torch.tensor(self.labels[idx])
    return item

Ahora se carga el tokenizador preentrenado de RoBERTuito, usando la clase AutoTokenizer

In [3]:
# GET TOKENIZER, VOCAB AND DIR OF DATASET--------------------------------------------------------------------------

from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("BSC-LT/mRoBERTa")

Dir = "../../corpus/proyecto"
vocab = tokenizer.get_vocab()

id2w = {}
for w in vocab:
    id2w[vocab[w]] = w

Finalmente, se crean las instancias de los datasets train y val

In [4]:
train_dataset = polar(Dir, 'train', tokenizer)
val_dataset   = polar(Dir, 'test'  , tokenizer, False)

## Modelo

Así como en HuggingFace se tiene la clase AutoModel, análogamente, en AdapterHub se tiene la clase AutoAdapterModel, con el que se puede cargar un modelo pre-entrenado de HuggingFace sin especificar la arquitectura, con la diferencia de que ya tiene todos los métodos para el uso de adapters incluidos

In [9]:
# Verificar que se instaló correctamente
import subprocess
import sys

# Listar paquetes instalados
!pip list | grep -i adapter

# Intentar importar
try:
    import adapter_transformers
    print("✅ adapter_transformers importado correctamente")

    # Mostrar versión
    import importlib.metadata
    version = importlib.metadata.version('adapter-transformers')
    print(f"📦 Versión: {version}")

except Exception as e:
    print(f"❌ Error: {e}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


adapter-transformers         4.0.0
adapters                     1.2.0
❌ Error: No module named 'adapter_transformers'


In [10]:
from adapters import AutoAdapterModel

model = AutoAdapterModel.from_pretrained("BSC-LT/mRoBERTa", num_labels = 2)

/home/juancho/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'Could not load this library: /home/juancho/.local/lib/python3.10/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at BSC-LT/mRoBERTa and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

## Adapter

Los modelos soportados por AdapterHub tienen el método "add_adapter"" que sirve para añadir un adapter al modelo. Principalmente tiene  dos parámetros:

*   **adapter_name: [string]** ,    usualmente nombre de la tarea que resuelve
*   **config: [str, dict, AdapterConfig]** ,    la arquitectura del adapter.

AdapterHub soporta diferentes arquitecturas para adapters de artículos recientes. Hay algunas configuraciones pre-establecidas como: PfeifferConfig(default), HoulsbyConfig, ParallelConfig, etc.. También hay alrededor de 400 adapters pre-entrenados que se pueden utilizar.

Cada **adapter** debe tener una **etiqueta de tipo string**, para referirse a él en el futuro, ya que un mismo modelo base puede tener más de un adapter. Si el adapter se utilizará para clasificación, debe añadirse también su respectiva **cabeza de clasificación con la misma etiqueta**.

In [ ]:
# name of the task
task_name = "mexA3"

# Add a new adapter
from transformers import ParallelConfig

model.add_adapter(
    adapter_name = task_name,
    config       = ParallelConfig()
)

# Add a matching classification head
model.add_classification_head(
    head_name  = task_name,
    num_labels = 2,
    id2label   = { 0: "Neutro", 1: "Agresivo"}
)

Hasta este punto, ya se añadió un adapter y una cabeza de clasificación. Sin embargo, como se puede ver en las siguientes celdas, todos los parámetros son entrenables.

In [ ]:
def trainable_parameters_relation(model):
  total_params = 0
  train_params = 0
  for name, param in model.named_parameters():
    curr = np.array(param.shape).prod()
    total_params += curr
    if param.requires_grad:
      #print(name)
      train_params += curr

  return 100*train_params/total_params

In [ ]:
print("\nParametros entrenables:", trainable_parameters_relation(model), "%")

En la siguiente celda se aplican dos operaciones:


*   La primera sirve para activar el adapter que vamos a utilizar, lo que obliga a que el input use las capas asociadas al adapter y a la cabeza de clasificación.
*   La segunda línea es NECESARIA para entrenar el adapter. Una de sus funciones es congelar los pesos del modelo base.



In [ ]:
# Activate the adapter
model.set_active_adapters(task_name)
model.train_adapter(task_name)

Se verifica nuevamente el procentaje de parámetros entrenables.

In [ ]:
print("\nParametros entrenables:", trainable_parameters_relation(model), "%")

## Entrenamiento

HuggingFace cuenta con una clase llamada Trainer, que sirve para entrenar sus modelos de forma sencilla en los casos más estándar (como en este caso, clasificación).

Antes de crear una instancia de Trainer, es necesario definir los parámetros que se utilizarán para el entrenamiento. Para ello, hay que crear una instancia de la clase TrainingArguments.

**Algunas ventajas**: cuida que el modelo y los datos estén en el mismo dispositivo (gpu, cpu) y, si hay más de 1 gpu, utiliza todos para hacer el entrenamiento en paralelo.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    learning_rate               = 1e-4,
    #weight_decay                 = 0.01,
    num_train_epochs            = 5,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size  = 32,
    logging_steps               = 100,
    output_dir                  = "./training_output",
    overwrite_output_dir        = True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns       = False,
)

Análogamente, AdapterHub tiene la clase AdapterTrainer, que está pensada para entrenar únicamente adapters. Se utiliza igual que Trainer.

Una de las ventajas es que si los pesos del modelo base no están congelados, al intentar entrenar con AdapterTrainer arrojará error.

In [ ]:
from transformers import AdapterTrainer, EvalPrediction

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model           = model,
    args            = training_args,
    train_dataset   = train_dataset,
    eval_dataset    = val_dataset,
    compute_metrics = compute_accuracy,
)
trainer.args._n_gpu = 1

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
model.save_adapter("weights", "mexA3")